
# Generando el dataset para el proyecto de clasificacion de noticias

Utilizamos los archivos csv proporcionados para crear nuestro dataset inicial (disponibles en noticias)

In [2]:
import pandas as pd
from os import listdir
from os.path import join
from tqdm import tqdm
from my_funcs import clean_and_tag
import re
from pandasql import sqldf
import numpy as np

In [3]:
pd.read_csv('./noticias/chile_2021-01.csv').head(3)

,id,country,media_outlet,url,title,text,date,year,id_journalist
0,3816243,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/ley-...,Ley de Migración: diputada Núñez (RN) formula ...,La diputada y vicepresidenta de Renovación Nac...,2021-01-01,2021.0,NaN
1,3816387,chile,elmostrador,https://www.elmostrador.cl/destacado/2021/01/0...,2021 arranca con 3.588 nuevos contagios y 52 f...,"Según el reporte de hoy, el Ministerio de Salu...",2021-01-01,2021.0,NaN
2,3816512,chile,elmostrador,https://www.elmostrador.cl/dia/2021/01/01/829-...,829 personas fueron detenidas durante noche de...,Carabineros reportó que hasta las 06:00 am. de...,2021-01-01,2021.0,NaN


 Ya que estas noticias entregadas no se encuentran etiquetadas es necesario realizar un procesamiento previo para asignarles una categoria que nos permita entrenar y probar nuestros modelos de clasificación.
 
 ### **Categorias a utilizar**: Se utilizarán las categorias propuestas en el enunciado del trabajo 
 
* 1- Mundo
* 2- Economía
* 3- Política y Conflictos
* 4- Ciencias y Tecnología,
* 5- Catástrofes y Accidentes,
* 6- Cultura y Artes, 
* 7- Deporte,
* 8- Ecología y Planeta,
* 9- Crimen, delitos y Justicia, 
* 10- Salud

### Adoptamos dos estrategias para realizar esta tarea:

1. Asignarle una categoria a partir de la información de la url.
2. Utilizar LDA para encontrar topicos que correspondan a alguna categoria.

# 1. Etiquetado a partir de la informacion contenida en la url

Debido a la gran cantidad de noticias, propusimos inicialmente utilizar esta metodologia para disminuir el trabajo inicial y aprovechando la propia información que nos proveen los medios de prensa.

Todas las noticias en su URL vienen con una categoria propia del medio, aunque esta pueda diferir entre los medios (una misma noticias para un medio puede ser *nacional* pero para otro puede ser *cultura*) podemos clasificar incialmente utilizando esta información.

In [4]:
unique_url_tags = set()
folder = './noticias'
dfs = []
for archivo in tqdm(listdir(folder)):
    df = pd.read_csv(join(folder, archivo))
    corpus = clean_and_tag(df)
    dfs.append(corpus)
    for tag in corpus.tag.unique():
        unique_url_tags.add(tag)

100%|██████████| 8/8 [00:02<00:00,  3.43it/s]


In [5]:
print(len(unique_url_tags))
list(unique_url_tags)[:10]

83


['tendencias',
 'noticiasbbc',
 'eclipse-2020',
 'mexico',
 'nacional',
 'tv',
 'regiones',
 'mercados',
 'animal',
 'mapuche']

### Encontramos 83 tags distintos. Revisamos esta lista y asignamos manualmente una categoria a los url_tags que pertenecieran a alguna de las 10 categorias objetivo (este trabajo manual se encuentra en el archivo ```categorias - hojav2.csv```)

In [6]:
dm = pd.read_csv('./categorias - hojav2.csv')
dm.rename(columns={'Unnamed: 0':'tag'}, inplace=True)
vals = dm.iloc[:, 1:]
labels = vals[vals.max(axis=1)>0].idxmax(axis=1)
print(len(labels))
labels.head(5)

40


1       8-Ecología_y_Planeta
2          6-Cultura_y_Artes
3    4-Ciencias_y_Tecnología
4    3-Política_y_Conflictos
8    4-Ciencias_y_Tecnología
dtype: object

### Utilizamos un diccionario para manejar de mejor manera las *tag_url* con las categorias propuestas

In [7]:
cats = pd.DataFrame(labels)
cats = cats.join(dm['tag'])
cats.rename(columns={0:'cat'}, inplace=True)
cats_dict = dict(zip(cats['tag'], cats['cat']))
list(cats_dict.items())[:5]

[('animal', '8-Ecología_y_Planeta'),
 ('artes', '6-Cultura_y_Artes'),
 ('Autos', '4-Ciencias_y_Tecnología'),
 ('braga', '3-Política_y_Conflictos'),
 ('ciencia-tecnologia', '4-Ciencias_y_Tecnología')]

Se agrega la columna ```category``` al dataframe rellenando con Nan. 
Creamos una columna *content* que contiene el titulo de la noticias más el cuerpo de esta.
Esto se realizó debido a que el titulo contiene información sumamente importante la cual explica la idea principal de esta en muy pocas lineas de texto, por lo que utilizarla nos proporciona información util para su clasificación.

In [8]:
df = pd.concat(dfs)
df.drop_duplicates(subset='url', keep='first', inplace=True)
df['category'] = df.apply(lambda row : cats_dict.get(row.tag, np.nan), axis=1)
#df['content'] = df['title'] + df['text']
df['content'] = df[['title', 'text']].agg('. '.join, axis=1)
df.head(3)

,id,country,media_outlet,url,title,text,date,tag,category,content
0,49234,chile,horas24,https://www.24horas.cl/coronavirus/tia-pikachu...,"""Tía Pikachu"" por parte empadronado: ""Estoy co...","Ha sufrido los embates del carro lanza aguas,...",2020-09-01,coronavirus,10-Salud,"""Tía Pikachu"" por parte empadronado: ""Estoy co..."
3,49291,chile,horas24,https://www.24horas.cl/nacional/rm-alcanza-sus...,RM alcanza sus mejores índices de calidad de a...,"La ministra del Medio Ambiente, Carolina Schm...",2020-09-01,nacional,NaN,RM alcanza sus mejores índices de calidad de a...
6,49335,chile,horas24,https://www.24horas.cl/nacional/paro-de-camion...,Paro de camioneros: Supermercados preocupados ...,"Esta jornada, la sexta de paralización de los...",2020-09-01,nacional,NaN,Paro de camioneros: Supermercados preocupados ...


Almacenamos 30.000 de las noticias cuyo *url_tag* era muy general o no nos servia para asignarle una categoria. A estas noticias en una segunda etapa le aplicaremos LDA  para poder etiquetarlas y asi aumentar la información de nuestro dataset inicial.

In [9]:
pd.set_option('display.max_colwidth', 120)
df = df[['id','url','title', 'content', 'category']]
unravel = df[df['category'].isna()]
(unravel.sample(n=30000, random_state=2)).to_csv('30k_sin_cat.csv')
df = df[df['category'].notna()]
final_df_link = df.drop(columns=['url'])
final_df_link.to_csv('final_df_link.csv')
final_df_link.head(3)

,id,title,content,category
0,49234,"""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre""","""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre"". Ha sufrido los embates del carro lanza...",10-Salud
7,49349,Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay,"Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay. Luego de la primera jornada de CyberDay,...",2-Economía
22,49598,"¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia""","¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia"". Este miércoles 2 de septiembre ...",10-Salud


Luego de esto, podemos ver que pudimos etiquetar de buena manera una cantidad considerable de noticias, pero, existen categorias las cuales no pudimos proveerles una gran cantidad de noticias y existieron otras, particularmente la categoria *5- Catástrofes y Accidentes* que no fue posible, esto es debido a que los medios de prensa no tienen en sus URL una categoria para catastrofes o accidentes.

Por otro lado de esta manera es posible que nuestro dataset este sesgado, por ejemplo, en la categoria de Salud, todas las noticias son apartir de Septiembre de 2020, fechas las cuales la gran mayoria de noticias son acerca de la pandemia del Coronavirus, por lo que noticias anteriores que quizás hablan de la influenza u otras enfermedades las cuales anteriormete eran comunes pero desde esta fecha no, no sea bien clasificadas por este metodo.


In [10]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


# 2. Aplicación de LDA a noticias no clasificadas

A las 30.000 noticias que guaradmos antes le aplicamos LDA con 20, creemos que es un buen número para poder apreciar una diferencia entre los grupos formados, debido a la gran cantidad de noticias a analizar

In [11]:
import re
from pprint import pprint
import nltk;
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import string
# spacy for lemmatization
import spacy
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
df = pd.read_csv('./30k_sin_cat.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head(3)

,id,url,title,content,category
0,6354016,https://www.latercera.com/que-pasa/noticia/vacuna-de-la-u-de-oxford-la-mas-avanzada-del-mundo-comenzara-sus-pruebas-...,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",NaN
1,26919,https://www.24horas.cl/nacional/comienza-la-postulacion-a-los-beneficios-estudiantiles-para-la-educacion-superior-20...,Comienza la postulación a los Beneficios Estudiantiles para la Educación Superior 2021,Comienza la postulación a los Beneficios Estudiantiles para la Educación Superior 2021. Este martes el Ministerio de...,NaN
2,165091,https://www.biobiochile.cl/noticias/nacional/region-de-los-lagos/2020/10/05/autoridades-regionales-de-los-lagos-preo...,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé. Desde el Consejo Regio...,NaN


In [13]:
def sent_to_words(sentences):
    for sentence in tqdm(sentences, desc="sentences to words"):
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [14]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
stop_words = stopwords.words('spanish')
nlp = spacy.load('es_core_news_md', disable=['parser', 'ner'])
def remove_stopwords(texts):
    desc = "removing stopwords"
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in tqdm(texts, desc=desc)]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in tqdm(texts, desc="making bigrams")]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in tqdm(texts, desc="making trigrams")]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm(texts, desc="lemmatization"):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
def get_ngrams(data_words):
    print('getting n-grams...')
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram, trigram, bigram_mod, trigram_mod

In [16]:
def get_lemmatized_data(df):
    data = df.content.values.tolist()
    data = [re.sub(r'\s+', ' ', sent) for sent in tqdm(data, desc="replacing whitespaces")]
    data_words = list(sent_to_words(data))
    bigram, trigram, bigram_mod, trigram_mod = get_ngrams(data_words)
    data_words_nostops = remove_stopwords(data_words)
    data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)
    data_lemmatized = lemmatization(data_words_bigrams,
                                    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    return data_lemmatized

In [17]:
import pickle
try:
    with open ('./data_lemmatized', 'rb') as fp:
        data_lemmatized = pickle.load(fp)
except FileNotFoundError:
    print('data_lemmatized not found...')
    data_lemmatized = get_lemmatized_data(df)
    with open('./data_lemmatized', 'wb') as fp:
        pickle.dump(data_lemmatized, fp)

data_lemmatized not found...


sentences to words: 100%|██████████| 30000/30000 [00:17<00:00, 1754.16it/s]


getting n-grams...


lemmatization: 100%|██████████| 30000/30000 [03:37<00:00, 137.86it/s]


### Entrenamiento del modelo de tópico con LDA

- Entrenaremos un primer modelo de tópicos buscando un modelo con 20 tópicos. 

In [18]:
model_file = "./models/model_lda_20_topics.mm"
num_topics = 20
#lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm", )

In [19]:
try:
    lda_model = gensim.models.ldamodel.LdaModel.load("./models/model_lda_20_topics.mm")
    id2word = corpora.Dictionary.load(model_file + ".id2word")  
except FileNotFoundError:
    id2word = corpora.Dictionary(data_lemmatized) 
    lda_model = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=num_topics,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True,
                                                   workers=4)
    lda_model.save(model_file)
texts = data_lemmatized
#corpus = [id2word.doc2bow(text) for text in tqdm(texts)]
corpus = [id2word.doc2bow(text) for text in tqdm(texts)]

100%|██████████| 30000/30000 [00:02<00:00, 13917.02it/s]


In [20]:
def format_topics_documents(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(tqdm(ldamodel[corpus])):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [21]:
try:
    df_dominant_topic = pd.read_csv('./df_dominant_topic.csv')
except FileNotFoundError:
    df_topic_sents_keywords = format_topics_documents(lda_model, corpus, texts)

    # Format
    df_dominant_topic = df_topic_sents_keywords.reset_index()
    df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'text_list']
    df_dominant_topic.to_csv('./df_dominant_topic.csv')
    df_dominant_topic.head(20)

100%|██████████| 30000/30000 [05:11<00:00, 96.21it/s] 


In [22]:
df_dominant_topic.to_csv('./df_dominant_topic.csv')

In [23]:
df_topics = pd.concat([df.drop(columns=['category']), df_dominant_topic[['Dominant_Topic', 'Keywords', 'text_list']]], axis=1)

In [24]:
df_topics['Dominant_Topic'] = df_topics['Dominant_Topic'].astype(int)
df_topics.head(3)

,id,url,title,content,Dominant_Topic,Keywords,text_list
0,6354016,https://www.latercera.com/que-pasa/noticia/vacuna-de-la-u-de-oxford-la-mas-avanzada-del-mundo-comenzara-sus-pruebas-...,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",4,"vacuna, salud, persona, dosis, primero, virus, paciente, decir, enfermedad, coronavirus","[vacuna, avanzado, mundo, anunciar, inicio, prueba, laboratorio, astrazeneca, anunciar, vacuna, mas, prometedor, mun..."
1,26919,https://www.24horas.cl/nacional/comienza-la-postulacion-a-los-beneficios-estudiantiles-para-la-educacion-superior-20...,Comienza la postulación a los Beneficios Estudiantiles para la Educación Superior 2021,Comienza la postulación a los Beneficios Estudiantiles para la Educación Superior 2021. Este martes el Ministerio de...,9,"colegio, ano, educacion, profesor, estudiante, clase, tambien, salud, alumno, nino","[comenzar, postulacion, beneficio, estudiantil, educacion_superior, martes, ministerio, educacion, iniciar, proceso,..."
2,165091,https://www.biobiochile.cl/noticias/nacional/region-de-los-lagos/2020/10/05/autoridades-regionales-de-los-lagos-preo...,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé. Desde el Consejo Regio...,17,"alcalde, sector, incendio, trabajador, vecino, municipio, común, trabajar, trabajo, bombero","[autoridad, regional, lago, preocupado, falta, regional, mostrar, falta, proteccion, fauna, archipielago, ano, cerca..."


# Análisis manual

In [25]:
from my_funcs import get_df_keywords
get_df_keywords(lda_model, num_topics)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,agua,region,caso,candidato,vacuna,proyecto,post_on,chileno,servicio,colegio,persona,presidente,hecho,ano,mujer,hacer,carabinero,alcalde,plebiscito,caso
1,ambiental,zona,publico,partido,salud,gobierno,instagrama_post,ano,producto,ano,paso,ministro,vehiculo,millón,menor,decir,persona,sector,voto,nuevo
2,pueblo,hora,delito,primario,persona,ministro,view_this,nuevo,persona,educacion,comuna,diputado,lugar,nuevo,genero,ver,policial,incendio,votar,contagio
3,comunidad,biobio,fiscal,independiente,dosis,ley,post_shared,primero,encontrar,profesor,medida,politico,hombre,mas,violencia,creer,general,trabajador,proceso,total
4,pueblos_originario,rio,fiscalia,candidatura,primero,diputado,shared_by,argentino,usuario,estudiante,fase,ex,encontrar,mayor,adolescente,querer,violencia,vecino,presidente,cifra
5,territorio,nacional,ministerio,alcalde,virus,presidente,ano,obra,sistema,clase,salud,cargo,persona,estudio,nino,gente,detenido,municipio,politico,salud
6,tierra,eclipse,hecho,acuerdo,paciente,hacer,by,mundo,contar,tambien,cuarentena,parlamentario,detenido,tambien,servicio,mas,hecho,común,electoral,persona
7,derecho,sismo,tribunal,ir,decir,parlamentario,hijo,serie,realizar,salud,plan,senador,ano,desarrollo,edad,persona,personal,trabajar,octubre,nivel
8,mapuche,total,ano,constituyente,enfermedad,comision,hacer,temporada,empresa,alumno,semana,jefe,joven,empresa,talcahuano,dar,institucion,trabajo,apruebo,aumento
9,isla,región,imputado,politico,coronavirus,retiro,vida,historia,hora,nino,permiso,decir,sujeto,nivel,familia,tambien,publico,bombero,votacion,región



    1- Mundo
    2- Economía
    3- Política y Conflictos
    4- Ciencias y Tecnología,
    5- Catástrofes y Accidentes,
    6- Cultura y Artes,
    7- Deporte,
    8- Ecología y Planeta,
    9- Crimen, delitos y Justicia,
    10- Salud


Analizando los 20 topicos determinamos que cada uno de estos corresponde a las siguentes categorias propuestas.

Algunos topicos como el 6, 13 y 19 serán procesados con LDA nuevamente para poder mejorar su clasificación debido a que en esta primera instancia no fue posible clasificarlos de buena manera.

In [26]:
cats_dict = {1 : '8-Ecología_y_Planeta',
             2 : '9-Crimen_delitos_y_Justicia',
             3 : '3-Política_y_Conflictos',
             4 : '10-Salud',
             5 : '3-Política_y_Conflictos',
             7 : '6-Cultura_y_Artes',
             11 : '3-Política_y_Conflictos',
             12 : '9-Crimen_delitos_y_Justicia',
             14 : '9-Crimen_delitos_y_Justicia',
             16 : '9-Crimen_delitos_y_Justicia',
             17 : '5-Catástrofes_y_Accidentes',
             18 : '3-Política_y_Conflictos',}

# el resto aun sin categoria
for i in range(0, num_topics):
    if i not in cats_dict.keys():
        cats_dict[i] = np.nan
        
cats_dict
df_topics['category'] = df_topics.apply(lambda row : cats_dict[row.Dominant_Topic], axis=1)
df_topics.head(10)

,id,url,title,content,Dominant_Topic,Keywords,text_list,category
0,6354016,https://www.latercera.com/que-pasa/noticia/vacuna-de-la-u-de-oxford-la-mas-avanzada-del-mundo-comenzara-sus-pruebas-...,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",4,"vacuna, salud, persona, dosis, primero, virus, paciente, decir, enfermedad, coronavirus","[vacuna, avanzado, mundo, anunciar, inicio, prueba, laboratorio, astrazeneca, anunciar, vacuna, mas, prometedor, mun...",10-Salud
1,26919,https://www.24horas.cl/nacional/comienza-la-postulacion-a-los-beneficios-estudiantiles-para-la-educacion-superior-20...,Comienza la postulación a los Beneficios Estudiantiles para la Educación Superior 2021,Comienza la postulación a los Beneficios Estudiantiles para la Educación Superior 2021. Este martes el Ministerio de...,9,"colegio, ano, educacion, profesor, estudiante, clase, tambien, salud, alumno, nino","[comenzar, postulacion, beneficio, estudiantil, educacion_superior, martes, ministerio, educacion, iniciar, proceso,...",NaN
2,165091,https://www.biobiochile.cl/noticias/nacional/region-de-los-lagos/2020/10/05/autoridades-regionales-de-los-lagos-preo...,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé. Desde el Consejo Regio...,17,"alcalde, sector, incendio, trabajador, vecino, municipio, común, trabajar, trabajo, bombero","[autoridad, regional, lago, preocupado, falta, regional, mostrar, falta, proteccion, fauna, archipielago, ano, cerca...",5-Catástrofes_y_Accidentes
3,4676443,https://www.latercera.com/nacional/noticia/etiquetas-de-valparaiso-que-luchan/ULMBF7T2EFASBK7NODY6WGYM3Y/,Etiquetas de Valparaíso que luchan,"Etiquetas de Valparaíso que luchan. Un milagroso bajativo de exportación, la clásica paleta de agua refrescante en l...",15,"hacer, decir, ver, creer, querer, gente, mas, persona, dar, tambien","[etiqueta, valparaiso, milagroso, bajativo, clasico, paletar, agua, refrescante, playa, rincon, colación, saludable,...",NaN
4,3834766,https://www.meganoticias.cl/nacional/323580-balance-covid-chile-cgx02.html,"Balance coronavirus: Minsal reporta 3.685 casos nuevos, cifra más alta de julio","Balance coronavirus: Minsal reporta 3.685 casos nuevos, cifra más alta de julio. El ministerio de Salud entregó este...",19,"caso, nuevo, contagio, total, cifra, salud, persona, nivel, aumento, región","[balance, coronavirus, minsal_reportar, caso, nuevo, cifra, alto, julio, ministerio, salud, entrego, jueves, nuevo, ...",NaN
5,4760120,https://www.latercera.com/nacional/noticia/camara-de-diputados-rechaza-por-falta-de-quorum-proyecto-de-educacion-sex...,Cámara de Diputados rechaza proyecto de educación sexual integral tras no alcanzar el quórum,Cámara de Diputados rechaza proyecto de educación sexual integral tras no alcanzar el quórum. La Cámara de Diputados...,5,"proyecto, gobierno, ministro, ley, diputado, presidente, hacer, parlamentario, comision, retiro","[rechazar, proyecto, educacion, sexual_integral, alcanzar, camarar, rechazo, faltar, quorum, voto, proyecto, estable...",3-Política_y_Conflictos
6,4382991,https://www.elmostrador.cl/dia/2021/01/08/el-alcalde-de-londres-declara-a-la-capital-en-situacion-grave-por-covid-19/,"El alcalde de Londres declara a la capital en situación ""grave"" por Covid-19","El alcalde de Londres declara a la capital en situación ""grave"" por Covid-19. El alcalde de Londres, el laborista Sa...",10,"persona, paso, comuna, medida, fase, salud, cuarentena, plan, semana, permiso","[alcalde, declarar, capital, situacion, grave, londr, laborista, declarado, hoy, capital, britanico, situacion, grav...",NaN
7,3994785,https://www.meganoticias.cl/nacional/317914-muerte-mujer-venezolana-paso-no-habilitado-colchane-cgx02.html,Migran

In [27]:
for i in range(num_topics):
    # agregar solo los category == nan
    if not isinstance(cats_dict[i], str) and np.isnan(cats_dict[i]):
        sub_df = df_topics[df_topics['Dominant_Topic'] == i]
        sub_df = sub_df[['id', 'title', 'content', 'Dominant_Topic', 'text_list']]
        sub_df.to_csv(f'./dominant_topics/dominant_topic{i}.csv')

In [28]:
final_df_topics = df_topics[['id','url','title', 'content', 'category']]
final_df_topics = final_df_topics[final_df_topics['category'].notna()]
final_df_topics.drop(columns=['url'], inplace=True)
final_df_topics.to_csv('final_df_topic.csv')
final_df_topics.head(3)

,id,title,content,category
0,6354016,"Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile","Vacuna de la U. de Oxford, la más avanzada del mundo, anuncia inicio de sus pruebas Fase 3 en Chile. La Universidad ...",10-Salud
2,165091,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé,Autoridades regionales de Los Lagos preocupadas por falta de protección a la fauna en Chiloé. Desde el Consejo Regio...,5-Catástrofes_y_Accidentes
5,4760120,Cámara de Diputados rechaza proyecto de educación sexual integral tras no alcanzar el quórum,Cámara de Diputados rechaza proyecto de educación sexual integral tras no alcanzar el quórum. La Cámara de Diputados...,3-Política_y_Conflictos


In [29]:
final_df_link = pd.read_csv('final_df_link.csv').drop(columns=['Unnamed: 0'])
final_df_link.head(3)

,id,title,content,category
0,49234,"""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre""","""Tía Pikachu"" por parte empadronado: ""Estoy complicada de aquí a diciembre"". Ha sufrido los embates del carro lanza...",10-Salud
1,49349,Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay,"Sernac recibió más de 400 reclamos durante la primera jornada de CyberDay. Luego de la primera jornada de CyberDay,...",2-Economía
2,49598,"¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia""","¿Cómo y dónde pueden abrir? Este miércoles reabren restaurantes en ""modo pandemia"". Este miércoles 2 de septiembre ...",10-Salud


In [30]:
final_df = pd.concat([final_df_link, final_df_topics]).reset_index().drop(columns=['index'])

In [31]:
q="""SELECT category, count(*) FROM final_df_topics GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,9-Crimen_delitos_y_Justicia,8034
1,3-Política_y_Conflictos,5734
2,10-Salud,1570
3,5-Catástrofes_y_Accidentes,1206
4,8-Ecología_y_Planeta,556
5,6-Cultura_y_Artes,363


In [32]:
q="""SELECT category, count(*) FROM final_df_link GROUP BY category ORDER BY count(*) DESC;"""
result=sqldf(q)
result

,category,count(*)
0,1-Mundo,8609
1,7-Deporte,6708
2,4-Ciencias_y_Tecnología,4225
3,2-Economía,2636
4,3-Política_y_Conflictos,1923
5,10-Salud,333
6,6-Cultura_y_Artes,185
7,8-Ecología_y_Planeta,135
8,9-Crimen_delitos_y_Justicia,130


# Clasificacion de subtopicos en el cuadernillo 2. ***paso_2_re_extraccion_lda.ipynb***